Parte 1: Generación del conjunto de datos

In [ ]:
import pandas as pd
import numpy as np
import unicodedata
import re
import os
import csv
import sklearn.metrics as metrics

In [ ]:
!pip install Sentencepiece
!pip install transformers

     |████████████████████████████████| 1.2 MB 12.0 MB/s 
     |████████████████████████████████| 3.5 MB 11.8 MB/s 
     |████████████████████████████████| 596 kB 46.7 MB/s 
     |████████████████████████████████| 6.8 MB 42.4 MB/s 
     |████████████████████████████████| 895 kB 45.4 MB/s 
     |████████████████████████████████| 67 kB 5.4 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
tsv_file = open("lcp_single_test.tsv")
read_tsv = csv.reader(tsv_file, delimiter="\t")

In [ ]:
label = 1
count = 0
data_raw = pd.DataFrame(columns = range(5))
for row in read_tsv:
  if label == 1:
    data_raw.columns = row
    label = 0
  else:
    data_raw.loc[count] = [row[0], row[1], row[2], row[3], row[4]]
    count += 1


In [ ]:
data_raw.head(10)

,id,corpus,sentence,token,complexity
0,3K8CQCU3KE19US5SN890DFPK3SANWR,bible,"But he, beckoning to them with his hand to be ...",hand,0.0
1,3Q2T3FD0ON86LCI41NJYV3PN0BW3MV,bible,"If I forget you, Jerusalem, let my right hand ...",hand,0.19736842105263158
2,3ULIZ0H1VA5C32JJMKOTQ8Z4GUS51B,bible,"the ten sons of Haman the son of Hammedatha, t...",hand,0.2
3,3BFF0DJK8XCEIOT30ZLBPPSRMZQTSD,bible,Let your hand be lifted up above your adversar...,hand,0.2678571428571429
4,3QREJ3J433XSBS8QMHAICCR0BQ1LKR,bible,"Abimelech chased him, and he fled before him, ...",entrance,0.0
5,30F94FBDNRKF12C1PDZGZ0IRI9XTB1,bible,"for as he thinks about the cost, so he is.",cost,0.25
6,3JHB4BPSFK9JLT5CAPRP3PCEA2CQ9R,bible,"When his armor bearer saw that Saul was dead, ...",armor,0.02777777777777779
7,3Y40HMYLL1I1EIURUEH8TTVLKQ9XUM,bible,"Therefore put on the whole armor of God, that ...",armor,0.2
8,30Y6N4AHYPWV0KXTZKLRUB59XBPRDA,bible,"Hezekiah listened to them, and showed them all...",armor,0.2222222222222222
9,38O9DZ0A62NFBR1TA7YCNXN94XN26F,bible,"That first slaughter, which Jonathan and his a...",armor,0.3194444444444444


In [ ]:
data = pd.DataFrame(columns=['text', 'complexity'])
count = 0
for i in range(0, data_raw.shape[0]):
  data.loc[count] = [data_raw['sentence'][count].lower() + ' [SEP] ' + data_raw['token'][count].lower(), \
                    data_raw['complexity'][count]]
  count += 1


In [ ]:
data['text'][0]

'but he, beckoning to them with his hand to be silent, declared to them how the lord had brought him out of the prison. [SEP] hand'

In [ ]:
# from transformers import BertTokenizer
# tokenizer = BertTokenizer.from_pretrained('bert-large-uncased')
from transformers import XLMRobertaTokenizer, XLMRobertaForSequenceClassification

MODEL_TYPE = 'xlm-roberta-base'

tokenizer = XLMRobertaTokenizer.from_pretrained(MODEL_TYPE)


Downloading:   0%|          | 0.00/4.83M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/8.68M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/512 [00:00<?, ?B/s]

In [ ]:
# Preprocess, tokenize and vectorize texts and labels
def vectorize_text(s, max_length):

    # Unicode normalization
    #s = s.strip().lower()
    s = ''.join(c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn')

    # Remove undesired characters
    #s = re.sub(r'([.¿?()\'",;:$€])', r' \1 ', s)
    #s = re.sub(r"[^a-zA-Záéíóú.,!?;:()$€]+", r" ", s)

    # Text to tensor
    input_ids = tokenizer.encode(
      s,
      add_special_tokens=True,
      max_length=max_length,
      padding='longest', 
      truncation=True,
      return_tensors='np'
    )
    return input_ids[0]

In [ ]:
data['text_vec'] = data.apply(lambda r: vectorize_text(r['text'], 512), axis=1)

In [ ]:
data

,text,complexity,text_vec
0,"but he, beckoning to them with his hand to be ...",0.0,"[0, 1284, 764, 4, 186, 60635, 592, 47, 2856, 6..."
1,"if i forget you, jerusalem, let my right hand ...",0.19736842105263158,"[0, 2174, 17, 90820, 398, 4, 55, 6563, 75040, ..."
2,"the ten sons of haman the son of hammedatha, t...",0.2,"[0, 70, 1492, 775, 7, 111, 256, 669, 70, 775, ..."
3,let your hand be lifted up above your adversar...,0.2678571428571429,"[0, 2633, 935, 3535, 186, 60520, 297, 1257, 36..."
4,"abimelech chased him, and he fled before him, ...",0.0,"[0, 18871, 32128, 206, 1608, 5281, 4049, 4, 13..."
...,...,...,...
882,"the report by mr philippe busquin, on behalf o...",0.5694444444444444,"[0, 70, 13416, 390, 33627, 44171, 150, 7340, 5..."
883,section v - court of auditors (sec(2002) 405 -...,0.5357142857142857,"[0, 40059, 81, 20, 29685, 111, 133604, 7, 15, ..."
884,"- mr president, on 1 june, on the basis of inf...",0.546875,"[0, 20, 33627, 13918, 4, 98, 106, 1129, 86, 4,..."
885,i would like to extend a warm welcome to this ...,0.5714285714285714,"[0, 17, 2806, 1884, 47, 65042, 10, 24814, 8190..."


In [ ]:
# Separar los conjuntos de datos
data = data.sample(frac=1)
train_portion = 0.9
split_point = int(train_portion*len(data))
train_data, test_data =  data[:split_point], data[split_point:] 
print(len(train_data), 'train, ', len(test_data), 'test')

798 train,  89 test


In [ ]:
#Create dataset en pytorch
from torch.utils.data import Dataset, DataLoader
import torch

class MyDataset(Dataset):
    def __init__(self, dataframe):
        self.len = len(dataframe)
        self.data = dataframe
        
    def __getitem__(self, index):
        input_ids = torch.tensor(self.data.text_vec.iloc[index]).cpu()
        attention_mask = torch.ones([input_ids.size(0)]).cpu()
        targets = self.data.complexity.iloc[index]
        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'labels': targets
         }
            
    def __len__(self):
        return self.len

In [ ]:
train_set, test_set = MyDataset(train_data), MyDataset(test_data)

In [ ]:
test_set.data

,text,complexity,text_vec
743,the return of cultural objects unlawfully remo...,0.2368421052631579,"[0, 70, 30646, 111, 15380, 36746, 7, 51, 19729..."
35,he gathered to him the children of ammon and a...,0.23214285714285715,"[0, 764, 914, 9319, 297, 47, 4049, 70, 20020, ..."
136,"yahweh will roar from zion, and thunder from j...",0.2833333333333333,"[0, 151, 67116, 127, 1221, 2062, 147, 1295, 10..."
449,"outside of the centromeric h3 variant, s. cere...",0.3035714285714286,"[0, 50782, 111, 70, 5863, 282, 12259, 1096, 36..."
326,"type ii fibers comprise three subtypes, iia, i...",0.2375,"[0, 10644, 17, 14, 109197, 7, 48402, 13, 17262..."
...,...,...,...
573,application of directive 2004/38/ec on the rig...,0.11764705882352944,"[0, 38415, 111, 8951, 5844, 4821, 18113, 74758..."
109,therefore you gave them into the hand of the p...,0.234375,"[0, 127298, 398, 26038, 2856, 3934, 70, 3535, ..."
141,"the arrow snake will make her nest there, and ...",0.35,"[0, 70, 6, 118201, 16854, 350, 1221, 3249, 604..."
400,mouse embryos that are homozygous for the gene...,0.25,"[0, 114669, 352, 21731, 232, 450, 621, 12840, ..."


In [ ]:
train_set.data

,text,complexity,text_vec
242,"hezro the carmelite, naarai the son of ezbai, ...",0.5125,"[0, 6, 6813, 516, 70, 2258, 27641, 67, 4, 2313..."
113,i find more bitter than death the woman whose ...,0.21875,"[0, 17, 7413, 1286, 114309, 3501, 47219, 70, 4..."
125,then david put garrisons in syria of damascus;...,0.3472222222222222,"[0, 7068, 48, 5518, 3884, 6, 29031, 42182, 23,..."
678,the contents of the report to be voted on toda...,0.2142857142857143,"[0, 70, 10941, 7, 111, 70, 13416, 47, 186, 433..."
816,after that we will have the council and commis...,0.3125,"[0, 7103, 450, 642, 1221, 765, 70, 215394, 136..."
...,...,...,...
746,the european council agreed that there is a ne...,0.25,"[0, 70, 98748, 215394, 176506, 450, 2685, 83, ..."
376,histopathological and ultrastructural studies ...,0.23214285714285715,"[0, 1919, 188, 128405, 109622, 136, 19914, 367..."
21,of all my sons (for yahweh has given me many s...,0.05357142857142855,"[0, 111, 756, 759, 775, 7, 15, 2472, 151, 6711..."
183,"its length was five cubits, its breadth was fi...",0.36363636363636365,"[0, 6863, 140909, 509, 43606, 123069, 933, 4, ..."


In [ ]:
len(test_set)

89

In [ ]:
len(train_set)
#len(test_set)

798

In [ ]:
# from transformers import BertForSequenceClassification, BertConfig
from transformers import XLMRobertaForSequenceClassification, XLMRobertaConfig
config = XLMRobertaConfig.from_pretrained('xlm-roberta-base')
config.problem_type='regression'
config.num_labels=1
model = XLMRobertaForSequenceClassification.from_pretrained('xlm-roberta-base', config=config)


Downloading:   0%|          | 0.00/1.04G [00:00<?, ?B/s]

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'roberta.pooler.dense.bias', 'lm_head.dense.bias', 'roberta.pooler.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.out_p

In [ ]:
from transformers import Trainer, TrainingArguments

def collate_batch(batch):
    """ Optimize memory by setting all vectors in batch to a length equal to max
        length found
    """
    
    def pad_sequence(in_tensor, max_size):
        """ Fill tensor with zeros up to max_size
        """
        out_tensor = np.zeros(max_size)
        out_tensor[:in_tensor.size(0)] = in_tensor.numpy()
        return out_tensor
    
    batch_inputs = []
    batch_attention_masks = []
    batch_targets = []

    max_size = max([ex['input_ids'].size(0) for ex in batch])
    for item in batch:
        batch_inputs.append(pad_sequence(item['input_ids'], max_size))
        batch_attention_masks.append(pad_sequence(item['attention_mask'], max_size))
        batch_targets.append(float(item['labels']))
    return {
        "input_ids": torch.tensor(batch_inputs, dtype=torch.long),
        "attention_mask": torch.tensor(batch_attention_masks, dtype=torch.long),
        "labels": torch.tensor(batch_targets, dtype=torch.float)
    }

class MyTrainer(Trainer):
  def __init__(self, **kwargs):
    super().__init__(**kwargs)

  def get_train_dataloader(self):
    return DataLoader(
        self.train_dataset,
        collate_fn=collate_batch
    )

  def get_eval_dataloader(self, eval_dataset):
    return DataLoader(
        self.eval_dataset,
        collate_fn=collate_batch
    )


In [ ]:
from transformers import EvalPrediction
from sklearn import metrics, feature_selection

## Para calcular nuestras propias métricas
def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions, tuple) else p.predictions
    preds = np.squeeze(preds)
    return {"MSE": ((preds - p.label_ids) ** 2).mean().item(),
            'R2': metrics.r2_score(p.label_ids, preds),
            'RMSE': metrics.mean_squared_error(p.label_ids, preds),
            'MAE': metrics.mean_absolute_error(p.label_ids, preds),
            'Poisson': metrics.mean_poisson_deviance(p.label_ids, preds)}
            #'Pearson': feature_selection.r_regression(p.label_ids, preds)}

In [ ]:
training_args = TrainingArguments(
    output_dir='output',
    evaluation_strategy='steps',
    eval_steps=100,
    num_train_epochs=5,
    remove_unused_columns=False,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
)

trainer = MyTrainer(
    model=model,
    args=training_args,
    train_dataset=train_set,
    eval_dataset=test_set,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 798
  Num Epochs = 5
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 3990
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:25: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:201.)


Step,Training Loss,Validation Loss,Mse,R2,Rmse,Mae,Poisson
100,No log,0.029103,0.029103,-0.833964,0.029103,0.124304,0.111020
200,No log,0.042695,0.042695,-1.690444,0.042695,0.167154,0.200158
300,No log,0.026116,0.026116,-0.645724,0.026116,0.143692,0.077807
400,No log,0.018988,0.018988,-0.196552,0.018988,0.114513,0.058789
500,0.045200,0.016552,0.016552,-0.043066,0.016552,0.100871,0.051571
600,0.045200,0.024532,0.024532,-0.545938,0.024532,0.138376,0.073764
700,0.045200,0.026324,0.026324,-0.658826,0.026324,0.115012,0.096054
800,0.045200,0.016542,0.016542,-0.042427,0.016542,0.100780,0.051540
900,0.045200,0.024862,0.024862,-0.566702,0.024862,0.110164,0.088595
1000,0.034800,0.015878,0.015878,-0.000577,0.015878,0.092552,0.049412


***** Running Evaluation *****
  Num examples = 89
  Batch size = 1
***** Running Evaluation *****
  Num examples = 89
  Batch size = 1
***** Running Evaluation *****
  Num examples = 89
  Batch size = 1
***** Running Evaluation *****
  Num examples = 89
  Batch size = 1
***** Running Evaluation *****
  Num examples = 89
  Batch size = 1
Saving model checkpoint to output/checkpoint-500
Configuration saved in output/checkpoint-500/config.json
Model weights saved in output/checkpoint-500/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 89
  Batch size = 1
***** Running Evaluation *****
  Num examples = 89
  Batch size = 1
***** Running Evaluation *****
  Num examples = 89
  Batch size = 1
***** Running Evaluation *****
  Num examples = 89
  Batch size = 1
***** Running Evaluation *****
  Num examples = 89
  Batch size = 1
Saving model checkpoint to output/checkpoint-1000
Configuration saved in output/checkpoint-1000/config.json
Model weights saved in output/checkpoint-10

TrainOutput(global_step=3990, training_loss=0.0279061025844182, metrics={'train_runtime': 1256.1582, 'train_samples_per_second': 3.176, 'train_steps_per_second': 3.176, 'total_flos': 105166394571900.0, 'train_loss': 0.0279061025844182, 'epoch': 5.0})